# Aprendizaje Computacional  

## Mario Graff (mgraffg@ieee.org, mario.graff@infotec.mx)  
## [https://github.com/ingeotec](https://github.com/ingeotec)
## [https://github.com/mgraffg](https://github.com/mgraffg)
## CONACYT - INFOTEC  

# Temas

1. Introducción
2. Aprendizaje supervisado
3. Métodos paramétricos
4. Métodos no-paramétricos
5. Máquinas de kernel
6. Métodos no convencionales de aprendizaje
7. Diseño y análisis de experimentos de aprendizaje
8. Aplicaciones

# Comparación de algoritmos

* Se tiene $k$ algoritmos
* ¿Quién es el mejor?

* El que entrena mejor - $\mathcal X$
* El que generaliza mejor - $\mathcal P$

# Validación cruzada

* Dividir $\mathcal X$ en dos conjuntos
* $\mathcal X$ y $\mathcal P$
* Estratificación

## Creación de $\mathcal X$ y $\mathcal P$

In [ ]:
from sklearn import datasets
import numpy as np
wine = datasets.load_wine()
mask = np.random.binomial(1, 0.7,
                          size=wine.data.shape[0]).astype(np.bool)
train = wine.data[mask]
test = wine.data[~mask]
test_target = wine.target[~mask]

# Rendimiento de algoritmos

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
linear = LinearSVC().fit(train, wine.target[mask]).predict(test)
logistic = LogisticRegression().fit(train, wine.target[mask]).predict(test)
naive = GaussianNB().fit(train, wine.target[mask]).predict(test)
print(accuracy_score(test_target, linear),
      accuracy_score(test_target, logistic),
      accuracy_score(test_target, naive))

# ¿Quién es mejor?

In [ ]:
res = []
for i in range(30):
    mask = np.random.binomial(1, 0.7, size=wine.data.shape[0]).astype(np.bool)
    train = wine.data[mask]
    test = wine.data[~mask]
    test_target = wine.target[~mask]
    linear = LinearSVC().fit(train, wine.target[mask]).predict(test)
    logistic = LogisticRegression().fit(train, wine.target[mask]).predict(test)
    naive = GaussianNB().fit(train, wine.target[mask]).predict(test)
    _ = [accuracy_score(test_target, linear), accuracy_score(test_target, logistic),
         accuracy_score(test_target, naive)]
    res.append(_)
res = np.array(res)

In [ ]:
np.mean(res, axis=0)

In [ ]:
%pylab inline
hist(res[:, 2])

In [ ]:
from scipy import stats
np.mean(res, axis=0), stats.sem(res, axis=0) * 1.96

# KFold

* Dividir $\mathcal X$ en $k$ conjuntos
* $\bigcap_i^k \mathcal X_i = \emptyset$
* Generar $k$ pares de ($\mathcal X$, $\mathcal P$)
* $A_1 = \mathcal X_1, \ldots, \mathcal X_{k-1}$
* $B_1 = \mathcal X_{k}$
* $A_2 = \mathcal X_1, \ldots, \mathcal X_{k-2}, \mathcal X_{k}$
* $B_2 = \mathcal X_{k-1}$

In [ ]:
from sklearn import model_selection
res = []
kf = model_selection.KFold(n_splits=30, random_state=0, shuffle=True)
for ts, vs in kf.split(wine.data):
    train = wine.data[ts]
    test = wine.data[vs]
    test_target = wine.target[vs]
    linear = LinearSVC().fit(train, wine.target[ts]).predict(test)
    logistic = LogisticRegression().fit(train, wine.target[ts]).predict(test)
    naive = GaussianNB().fit(train, wine.target[ts]).predict(test)
    _ = [accuracy_score(test_target, linear), accuracy_score(test_target, logistic),
         accuracy_score(test_target, naive)]
    res.append(_)
res = np.array(res)
np.mean(res, axis=0)

# Diferencias en rendimiento

from sklearn import model_selection

In [ ]:
from scipy.stats import kruskal
kruskal(res[:, 0], res[:, 1], res[:, 2])

# Comparación de dos algoritmos

In [ ]:
from scipy.stats import wilcoxon
wilcoxon(res[:, 0], res[:, -1])

# Comparación de $k$ algoritmos contra el mejor

In [ ]:
from EvoMSA.command_line import CommandLinePerformance
CommandLinePerformance.compute_p(res)

# Medidas de rendimiento para clasificación

# _Accuracy_

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
target = np.random.randint(0, 3, size=100)
hy = np.random.randint(0, 3, size=100)
np.mean(target == hy), accuracy_score(target, hy)

# ¿Qué tan bueno es el _accuracy_?

In [ ]:
target = np.zeros(100)
target[-10:] = 1
hy = np.zeros(100)
np.mean(target == hy)

# _Recall_

* Usado en recuperación de información
* Del total de objetos buscados ¿Cuál es la proporción que recuperaste?

In [ ]:
from sklearn.metrics import recall_score
def recall(target, hy):
    res = []
    for k in np.unique(target):
        m = (hy == k) & (target == k)
        res.append(m.sum() / (target == k).sum())
    return res    

In [ ]:
print(recall_score(target, hy, average=None))
print(recall(target, hy))

In [ ]:
mask = np.random.binomial(1, 0.7, size=wine.data.shape[0]).astype(np.bool)
train = wine.data[mask]
test = wine.data[~mask]
test_target = wine.target[~mask]
linear = LinearSVC().fit(train, wine.target[mask]).predict(test)
print(recall_score(test_target, linear, average=None))

In [ ]:
print(recall(test_target, naive))

# Precision

* La proporción de ejemplos correctamente etiquetados

In [ ]:
target = np.zeros(100)
target[50:] = 1
hy = np.ones(100)
hy[0] = 0

In [ ]:
accuracy_score(target, hy), recall_score(target, hy, average=None)

In [ ]:
def precision(target, hy):
    res = []
    for k in np.unique(target):
        mask = hy == k
        res.append((target[mask] == k).sum() / mask.sum())
    return res

In [ ]:
from sklearn.metrics import precision_score
precision_score(target, hy, average=None), precision(target, hy)

In [ ]:
precision_score(test_target, naive, average=None), precision(test_target, naive)

# F1

* Media armónica entre _recall_ y _precision_
* $2 \frac{p \cdot r}{p + r}$

In [ ]:
from sklearn.metrics import f1_score
f1_score(target, hy, average=None)

# Macro

* Macro-recall
* Macro-precision
* Macro-F1

In [ ]:
f1_score(target, hy, average='macro')

# AUC

* Area bajo la curva ROC

In [ ]:
%pylab inline
from sklearn.metrics import roc_curve
from sklearn import datasets
bc= datasets.load_breast_cancer()
mask = np.random.binomial(1, 0.7, size=bc.data.shape[0]).astype(np.bool)
train = bc.data[mask]
test = bc.data[~mask]
test_target = bc.target[~mask]
m = GaussianNB().fit(train, bc.target[mask])
df = m.predict_proba(test)
fpr, tpr, _ = roc_curve(test_target, df[:, 1] - df[:, 0])
plot(fpr, tpr)
plot([0, 1], [0, 1], color='navy', linestyle='--')